In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import random

In [ ]:
from tqdm import tqdm
from skimage.io import imread,imshow
from skimage.transform import resize
seed=12
np.random.seed = seed

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c data-science-bowl-2018

In [ ]:
!mkdir stage1_test
! unzip stage1_test.zip -d stage1_test
!mkdir stage1_train
! unzip stage1_train.zip -d stage1_train

In [ ]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH='/content/stage1_train/'
TEST_PATH='/content/stage1_test/'

#read all the id's from the training images
train_ids = next(os.walk(TRAIN_PATH))[1] 
test_ids = next(os.walk(TEST_PATH))[1]  # [1] means list of subfolder


In [ ]:

X_train = np.zeros((len(train_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)
Y_train = np.zeros((len(train_ids),IMG_HEIGHT,IMG_WIDTH,1),dtype=np.bool)


In [ ]:
print("Resizing training images and masks ")
for n,id_ in tqdm(enumerate(train_ids),total=len(train_ids)):
  path = TRAIN_PATH + id_
  # print(path)
  # print(id_)
  img = imread(path + '/images/' + id_ +'.png')[:,:,:IMG_CHANNELS]
  # imshow(img)
  # if n==1:
  #   break
  # print(img)
  # print(img.shape)
  img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range=True)
  X_train[n]=img # fill empty X_train values from img
  # for mask
  mask = np.zeros((IMG_HEIGHT,IMG_WIDTH,1),dtype=np.bool)
  for mask_file in next(os.walk(path + '/masks/'))[2]:
    mask_ = imread(path + '/masks/' + mask_file)
    mask_ = np.expand_dims(resize(mask_,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range=True),axis=-1) # adding new axis
    mask = np.maximum(mask_,mask)
  Y_train[n] = mask
#  this way we put all the mask of one image put together and make a one separate image



In [ ]:
X_test = np.zeros((len(test_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.uint8)
sizes_test = []
print("Resizing test images ")
for n,id_ in tqdm(enumerate(test_ids),total=len(test_ids)):
  path = TEST_PATH + id_
  img = imread(path + '/images/' + id_ +'.png')[:,:,:IMG_CHANNELS]
  sizes_test.append([img.shape[0],img.shape[1]])
  img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode='constant',preserve_range=True)
  X_test[n]=img # fill empty X_test values from img
  
print("Done...!")



In [ ]:
image_x = random.randint(0,len(train_ids))
imshow(X_train[image_x])
plt.show()
imshow(np.squeeze(Y_train[image_x]))
plt.show()


In [ ]:
# build the model
inputs = tf.keras.layers.Input((IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))


In [ ]:
# because keras layers Conv2D take floating value so we convert our input into floating value
s=tf.keras.layers.Lambda(lambda x: x/255)(inputs)

c1 = tf.keras.layers.Conv2D(16,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2,2))(c1)


c2 = tf.keras.layers.Conv2D(32,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2,2))(c2)

c3 = tf.keras.layers.Conv2D(64,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2,2))(c3)

c4 = tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D((2,2))(c4)

c5 = tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256,(3,3),activation= 'relu',kernel_initializer= 'he_normal',padding='same')(c5)

# upsampling

u6 = tf.keras.layers.Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6,c4])
c6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7,c3])
c7 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c7)

u8 = tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8,c2])
c8 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16,(2,2),strides=(2,2),padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9,c1],axis=3)
c9 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16,(3,3),activation='relu',kernel_initializer='he_normal',padding='same')(c9)

outputs = tf.keras.layers.Conv2D(1,(1,1),activation = 'sigmoid')(c9)

model = tf.keras.Model(inputs=[inputs],outputs=[outputs])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
# model.compile(optimizer='RMSprop',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()
#  becuase it is binary classification either it is a cell or not a cell so we use binary crossentropy



#

In [ ]:
################################################
#model check point
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.h5',verbose=1,save_best_only=True)
callbacks=[
           tf.keras.callbacks.EarlyStopping(patience=2,monitor='val_loss'),
           tf.keras.callbacks.TensorBoard(log_dir='logs')
           ]

history = model.fit(X_train,Y_train,validation_split=0.1,batch_size=16,epochs=25,callbacks=callbacks)


In [ ]:
#############################################
idx = random.randint(0,len(X_train))

preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)],verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):],verbose=1)
preds_test = model.predict(X_test, verbose=1)


preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# sanity check on some random training samples
ix = random.randint(0,len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

# sanity check on some random validation samples
ix = random.randint(0,len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()




In [ ]:
ix = random.randint(0,len(preds_test_t))
imshow(X_test[ix])
plt.show()
imshow(np.squeeze(preds_test_t[ix]))
plt.show()

In [ ]:
# !tensorboard --logdir=logs/ --host localhost --port 5000

In [ ]:
acc =history.history['accuracy']
val_acc = history.history['val_accuracy']
loss=history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs,acc,'b',label='Training Accuracy')
plt.plot(epochs,val_acc,'r',label='Validation Accuracy')
plt.xlabel("Number of Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation accuracy")
plt.legend()
plt.savefig("T_V_accuracy_rms.png")

plt.figure()
plt.plot(epochs,loss,'b',label='Training Loss')
plt.plot(epochs,val_loss,'r',label='Validation Loss')
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("T_V_loass_rms.png")
plt.show()